# WaveTrade Main Code

1- Install and import all the necessary libraries

2- Mount Google Drive

3- Pip install Pillow

4- Import Pillow

5- Define paths and create folders for the existing image repository on Google Drive

6- Define a folder to store captured screenshots

7- Initialize variables to store the last capture time

8- Define helper functions to capture a screenshot and preprocess a screenshot

9- Define the image data generator for augmentation

10- Create a generator for the images in the folder

11- Create an empty list to store the augmented images

12- Loop through each image in the generator and augment it

13- Resize the images so they're all the same size

14- Define the convolutional neural network model

15- Retrieve and preprocess live screenshots continuously

16- Make predictions using the trained model and decide whether to buy or sell

17 & 18- Calculate similarity between two images using SSIM
Main loop (capture and process screenshots)

In [ ]:
# Step 1: Install and import all the necessary libraries
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive
import cv2
import time
import os
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage import io
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


In [ ]:
# Step 2: Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Step 3
!pip install Pillow


In [ ]:
# Step 4
from PIL import Image

In [ ]:
#Step 5

# Path to the folder containing the images
folder_path = '/content/drive/MyDrive/WaveTrade/Candlestick_Pattern_library'

# Get the list of all files in the folder
file_list = os.listdir(folder_path)

# Filter the list to keep only image files
image_files = [file for file in file_list if file.endswith(('.jpg', '.jpeg', '.png'))]

# Print the number of images in the folder
print(f"Number of images in the folder: {len(image_files)}")


Number of images in the folder: 130


Define paths and create folders

In [ ]:
# Step 6
# Path to the existing image repository
image_repo_path = '/content/drive/MyDrive/WaveTrade/Candlestick_Pattern_library'

# Create a folder to store captured screenshots
captured_screenshots_folder = '/content/captured_screenshots'
os.makedirs(captured_screenshots_folder, exist_ok=True)


Initialize variables

In [ ]:
# Step 7
# Initialize a variable to store the last capture time
last_capture_time = 0


Define helper functions

In [ ]:
# Step 8
# Function to capture a screenshot (replace with actual code)
def capture_screenshot():
    # Replace this with your code to capture a screenshot
    pass

# Function to preprocess a screenshot (modify as needed)
def preprocess_screenshot(screenshot):
    # Preprocess the screenshot as per your requirements
    # Ensure that you return the image; otherwise, it will result in a None
    pass

# Function to calculate similarity between two images using SSIM
def calculate_similarity(image1, image2):
    return ssim(image1, image2, multichannel=True)


Define the image data generator for augmentation of the images

In [ ]:

# Step 9
# Define the path to the folder containing original images
folder_path = '/content/drive/MyDrive/WaveTrade/Candlestick_Pattern_library'

# Define the image data generator for augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


Create a generator for the images in the folder

In [ ]:
# Step 10
# Create a generator for the images in the folder
image_generator = datagen.flow_from_directory(
    folder_path,
    target_size=(224, 224),
    batch_size=1,
    class_mode=None,
    shuffle=False
)


Found 0 images belonging to 1 classes.


Create an empty list to store the agumented images

In [ ]:
# Step 11
# Create an empty list to store the augmented images
augmented_images = []


Resize the image so they're all the same size

In [ ]:

# Step 13
# Target size for resizing
target_size = (224, 224)

# Loop through each image in the folder and resize it
for image_file in image_files:
    # Create the full file path
    file_path = os.path.join(folder_path, image_file)

    # Open the image using Pillow
    image = Image.open(file_path)

    # Resize the image
    image = image.resize(target_size)

    # Save the resized image back to the same file
    image.save(file_path)

print("All images have been resized to:", target_size)


All images have been resized to: (224, 224)


Define the convolutional neural network model

In [ ]:
# Setp 14
# Create the AlexNet model
def create_alexnet_model(input_shape, num_classes):
    model = Sequential()

    # Layer 1: Convolutional Layer with 96 filters, each of size 11x11, and a stride of 4
    model.add(Conv2D(96, kernel_size=(11, 11), strides=(4, 4), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

    # Layer 2: Convolutional Layer with 256 filters, each of size 5x5
    model.add(Conv2D(256, kernel_size=(5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

    # Layer 3: Convolutional Layer with 384 filters, each of size 3x3
    model.add(Conv2D(384, kernel_size=(3, 3), activation='relu'))

    # Layer 4: Convolutional Layer with 384 filters, each of size 3x3
    model.add(Conv2D(384, kernel_size=(3, 3), activation='relu'))

    # Layer 5: Convolutional Layer with 256 filters, each of size 3x3
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

    # Flatten the output of the previous layer to feed into the fully connected layers
    model.add(Flatten())

    # Layer 6: Fully Connected Layer with 4096 neurons
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))

    # Layer 7: Fully Connected Layer with 4096 neurons
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))

    # Layer 8: Fully Connected Layer with num_classes neurons (output layer)
    model.add(Dense(num_classes, activation='softmax'))

    return model

# Define the input shape and number of classes
input_shape = (224, 224, 3)  # Adjust the input shape based on your resized image size
num_classes = 2  # Adjust the number of classes based on your classification task (bullish and bearish)

# Create the model
model = create_alexnet_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 54, 54, 96)        34944     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 26, 26, 96)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 256)       614656    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 10, 10, 256)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 8, 384)         885120    
                                                                 
 conv2d_3 (Conv2D)           (None, 6, 6, 384)         1

Retrieve and preprocess live screenshots continuously

In [ ]:
# Step 15
# Retrieve and preprocess live screenshots continuously
def capture_screenshot():
    # Implement code to capture a live screenshot using pyautogui or any other library
    pass

def preprocess_screenshot(screenshot):
    # Implement code to preprocess the captured screenshot
    pass


Make predictions using the trained model and decide whether to buy or sell

In [ ]:
# Step 16
# Make predictions using the trained model and decide whether to buy or sell
while True:
    # Capture a live screenshot
    screenshot = capture_screenshot()

    # Preprocess the captured screenshot and convert to a NumPy array
    preprocessed_screenshot = preprocess_screenshot(screenshot)

    # Check if the preprocessing is successful and not None
    if preprocessed_screenshot is not None:
        # Make predictions using the trained model
        prediction = model.predict(np.expand_dims(preprocessed_screenshot, axis=0))

        # Decide whether to buy or sell based on the prediction
        if prediction > 0.5:
            print("Buy")
        else:
            print("Sell")
    else:
        print("Preprocessing failed. Unable to make prediction.")


Preprocessing failed. Unable to make prediction.
Preprocessing failed. Unable to make prediction.
Preprocessing failed. Unable to make prediction.
Preprocessing failed. Unable to make prediction.
Preprocessing failed. Unable to make prediction.
Preprocessing failed. Unable to make prediction.
Preprocessing failed. Unable to make prediction.
Preprocessing failed. Unable to make prediction.
Preprocessing failed. Unable to make prediction.
Preprocessing failed. Unable to make prediction.
Preprocessing failed. Unable to make prediction.
Preprocessing failed. Unable to make prediction.
Preprocessing failed. Unable to make prediction.
Preprocessing failed. Unable to make prediction.
Preprocessing failed. Unable to make prediction.
Preprocessing failed. Unable to make prediction.
Preprocessing failed. Unable to make prediction.
Preprocessing failed. Unable to make prediction.
Preprocessing failed. Unable to make prediction.
Preprocessing failed. Unable to make prediction.
Preprocessing failed

KeyboardInterrupt: ignored

Main Loop (Capture and Process Screenshots)

In [ ]:
# Step 17 & 18
while True:
    # Capture a live screenshot
    screenshot = capture_screenshot()

    # Preprocess the captured screenshot and convert to a NumPy array
    preprocessed_screenshot = preprocess_screenshot(screenshot)

    # Check if the preprocessing is successful and not None
    if preprocessed_screenshot is not None:
        # Capture a screenshot every 15 seconds
        current_time = time.time()
        if current_time - last_capture_time >= 15:
            last_capture_time = current_time

            # Save the captured screenshot to the folder
            screenshot_path = os.path.join(captured_screenshots_folder, f'screenshot_{int(current_time)}.png')
            cv2.imwrite(screenshot_path, preprocessed_screenshot)

            # Measure similarity with images in the repository
            for image_file in os.listdir(image_repo_path):
                image_path = os.path.join(image_repo_path, image_file)
                existing_image = io.imread(image_path)
                similarity = calculate_similarity(existing_image, preprocessed_screenshot)

                # Define a threshold for similarity (adjust as needed)
                similarity_threshold = 0.8

                # Check if the similarity is above the threshold
                if similarity > similarity_threshold:
                    print(f"Similarity: {similarity:.2f} - Buy")
                else:
                    print(f"Similarity: {similarity:.2f} - Sell")

                # Delete the captured screenshot
                os.remove(screenshot_path)

    else:
        print("Preprocessing failed. Unable to make prediction.")

    # Sleep for a while before capturing the next screenshot
    time.sleep(1)


Preprocessing failed. Unable to make prediction.
Preprocessing failed. Unable to make prediction.
Preprocessing failed. Unable to make prediction.
Preprocessing failed. Unable to make prediction.
Preprocessing failed. Unable to make prediction.
Preprocessing failed. Unable to make prediction.
Preprocessing failed. Unable to make prediction.
Preprocessing failed. Unable to make prediction.
Preprocessing failed. Unable to make prediction.
Preprocessing failed. Unable to make prediction.
Preprocessing failed. Unable to make prediction.


KeyboardInterrupt: ignored

Notes:

Have a screen shot happen every 10-15 seconds and put it in a folder.  There should only ever be one image in the folder.  Measure images in the library against the screen shot in the folder.  If the accuracy is 85% or higher, make the buy/sell, otherwise no action.  Delete the image in the folder after the decision is made and the image is retrieved from the folder.

You need to havea cell that measures what the similarity rating is.  Look again at how the accuracy is being measured on the google image recognizer.

Also have an option for nothing to happen along side the buy and sell options.

Can also allow for a user to take a screenshot and put into the folder to validate the code working.

# Nice to Have

Augment the images
